## Customer Churn Model Scoring
#### The objectives of this lab is:
- score **new** customer data against a pre-built model
- schedule the notebook to run via the Notebook scheduler

### Step 1: Download new customer data

In [1]:
import wget
url_customer='https://raw.githubusercontent.com/yfphoon/dsx_demo/master/data/new_customer_churn_data.csv'

#remove existing files before downloading
!rm -f new_customer_churn_data.csv

customerFilename=wget.download(url_customer)

!ls -l new_customer_churn_data.csv

-rw------- 1 s3b2-c7634938ff52ab-a5f39cf201a0 users 27597 Nov 13 09:17 new_customer_churn_data.csv


### Step 2: Read data into a Spark DataFrame
**Note**: the new dataset does not contain the label column

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

newData= sqlContext.read\
    .format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(customerFilename)

In [3]:
newData = newData.withColumnRenamed("Est Income", "EstIncome").withColumnRenamed("Car Owner","CarOwner")
newData.toPandas().head()

,ID,Gender,Status,Children,EstIncome,CarOwner,Age,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,Usage,RatePlan
0,2048,F,S,1,13576.5,N,39.426667,14.83,0,25.66,0,CC,Budget,Standard,40.49,1
1,2054,F,M,2,84166.1,N,54.013333,3.28,0,11.74,1,CC,Budget,Standard,15.02,2
2,2075,F,S,0,68427.4,N,42.393333,23.76,0,50.05,0,Auto,FreeLocal,Standard,73.81,3
3,2095,F,M,2,77551.1,Y,33.600000,20.53,0,41.89,1,CC,Budget,Intnl_discount,62.42,2
4,2108,F,S,1,13109.1,N,62.606667,22.38,0,40.48,0,Auto,Budget,Standard,62.87,1


### Step 3: Load Saved Model

In [4]:
from pyspark.ml import PipelineModel
model1_loaded = PipelineModel.load("PredictChurn.churnModel")

### Step 4: Score the new data
Note: The scored output contains the predicted values and confidence scores

In [5]:
result = model1_loaded.transform(newData)

### Step 5: Export Score into a csv file

In [6]:
#Select ID, prediction and probability fields from the result dataframe

r1=result.select(result["ID"],result["predictedLabel"],result["prediction"],result["probability"])
r1.show(5,False)

+----+--------------+----------+-----------------------------------------+
|ID  |predictedLabel|prediction|probability                              |
+----+--------------+----------+-----------------------------------------+
|2048|T             |1.0       |[0.017391304347826087,0.9826086956521738]|
|2054|T             |1.0       |[0.39325700280112047,0.6067429971988795] |
|2075|F             |0.0       |[0.942270779314416,0.05772922068558384]  |
|2095|F             |0.0       |[0.9763728885493592,0.023627111450640864]|
|2108|T             |1.0       |[0.075,0.925]                            |
+----+--------------+----------+-----------------------------------------+
only showing top 5 rows



#### Decompose the probability column
The probability column contains a vector for each record, and the elements must be extracted

In [7]:
from pyspark.sql import Row
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors

udf_0 = udf(lambda vector: float(vector[0]), DoubleType())
udf_1 = udf(lambda vector: float(vector[1]), DoubleType())

r2 = (r1.select(r1["ID"], r1["prediction"],r1["probability"])
    .withColumn('probability_0', udf_0(r1.probability))
    .withColumn('probability_1', udf_1(r1.probability))
    .drop("probability"))

r2.show(10, False)

+----+----------+--------------------+--------------------+
|ID  |prediction|probability_0       |probability_1       |
+----+----------+--------------------+--------------------+
|2048|1.0       |0.017391304347826087|0.9826086956521738  |
|2054|1.0       |0.39325700280112047 |0.6067429971988795  |
|2075|0.0       |0.942270779314416   |0.05772922068558384 |
|2095|0.0       |0.9763728885493592  |0.023627111450640864|
|2108|1.0       |0.075               |0.925               |
|2124|0.0       |0.9919057692664552  |0.00809423073354488 |
|2154|1.0       |0.16981225296442687 |0.8301877470355731  |
|2218|0.0       |0.9143637805237788  |0.08563621947622133 |
|2267|0.0       |0.9805743411792578  |0.01942565882074223 |
|2284|1.0       |0.08849206349206348 |0.9115079365079366  |
+----+----------+--------------------+--------------------+
only showing top 10 rows

